# Atomic structure analysis with radial (azimuthal) average & variance profiles
### Jinseok Ryu (jinseok.ryu@diamond.ac.uk)
Only compatible with the ePSIC data processig workflow  
Recommended to run this notebook using Python 3.10 EPSIC kernel on the jupyterhub of Diamond Light Source  
Otherwise, it is highly probable that it will not work properly  
[Required Python packages]  
scipy, numpy, matplotlib, py4DSTEM, hyperspy, drca (optional for NMF, https://github.com/jinseuk56/drca)

In [ ]:
import sys
sys.path.append('/dls/science/groups/e02/Ryu/RYU_at_ePSIC/massive_radial_analysis')

from radial_profile_analysis import *

In [ ]:
# load data
base_dir = '/dls/e02/data/2024/mgXXXXX-1/processing/Merlin/'
subfolders = [''] # subfolder names you want to load and compare

profile_length = 360 # limit the profile size
num_load = 100 # limit the number of data for every subfolder (select files randomly)

include_key = [] # keyword for screening (to only include the specified data)
exclude_key = [] # keyword for screening (to exclude poor quality data)

run_analysis = radial_profile_analysis(base_dir, subfolders, 
                                       profile_length, num_load, 
                                       include_key, exclude_key, verbose=True)

In [ ]:
# Transformation quality check (center beam alignment)
# If there are any data of poor quality, you can exclude them in the cell above (using 'exclude_key')
# crop=[top, bottom, left, right] -> img[top:bottom, left:right]
run_analysis.center_beam_alignment_check(crop=[220, -150, 220, -150])

In [ ]:
# Bright-field image
run_analysis.intensity_integration_image()

In [ ]:
# Specify the scattering vector range
# Simulate diffraction patterns (XRD) from CIF (optional])
str_path = ['/dls/science/groups/e02/Ryu/RYU_at_ePSIC/massive_radial_analysis/structure_file/structure_file_YC/Li2O.cif',
           '/dls/science/groups/e02/Ryu/RYU_at_ePSIC/massive_radial_analysis/structure_file/structure_file_YC/Li3P.cif',
           '/dls/science/groups/e02/Ryu/RYU_at_ePSIC/massive_radial_analysis/structure_file/structure_file_YC/LiP.cif'] # structure paths to compare

from_unit = 0.1 # unit: 1/angstrom, it must be equal to or greater than zero
to_unit = 1.0 # unit: 1/angstrom, it must be smaller than the maximum scattering vector
run_analysis.basic_setup(str_path, from_unit, to_unit, broadening=0.01) # broadening -> used to simulate diffraction patterns

In [ ]:
# Sum of radial variance and average profiles
# profile_type: "mean" or "variance"
# str_name=["structure_name_1", "structure_name_2"]
run_analysis.sum_radial_profile(str_name=["Li2O"], 
                                profile_type="variance")

In [ ]:
# NMF with normalization (rescale every profile from 0 to 1)
# Please refer to Scikit-learn, 'nmf' or 'https://github.com/jinseuk56/drca'
# profile_type: "mean" or "variance"
num_comp = 5
run_analysis.NMF_decompose(num_comp, 
                           max_normalize=False, 
                           rescale_0to1=True, 
                           profile_type="variance", 
                           verbose=True)

In [ ]:
# NMF - loading vectors and their coefficient maps
run_analysis.NMF_result()

In [ ]:
# NMF - show the pixels with high coefficients for each loading vector and the averaged profiles for those pixels
# str_name=["structure_name_1", "structure_name_2"]
# percentile_threshold -> if 90, only the pixels with the 10% highest coefficients remain
run_analysis.NMF_comparison(str_name=["Li2O"], 
                            percentile_threshold=90)

In [ ]:
# Peak detection
# Please refer to SciPy 'find_peaks' for details
# scattering vector range -> [peak_position-half_width, peak_position+half_width]
half_width = 0.01
run_analysis.scattering_range_of_interest(profile_type="variance",
                                          str_name=["Li2O"],
                                         fill_width=half_width,
                                         prominence=0.001,
                                         height=None,
                                         width=None,
                                         distance=None,
                                         threshold=None)

In [ ]:
# Variance maps for the specified scattering vector range
# Average and standard deviation of variances for the specified scattering vector range
peak_selected = 0.62
run_analysis.variance_map(sv_range=[peak_selected-half_width, peak_selected+half_width])

In [ ]:
# Boolean map (the pixels with higher variances than the absolute threshold)
run_analysis.high_variance_map(abs_threshold=0.03)

In [ ]:
# Calculate the cross correlation between radial profiles and a simulated diffraction pattern
# Please refer to Numpy, 'correlate'
run_analysis.Xcorrel(str_name="Li2O", profile_type="mean")

In [ ]:
# Show the pixels with high cross correlation values and low shifts (lags)
run_analysis.high_Xcorr(value_threshold=0.5, shift_threshold=0.5)

In [ ]:
# Summary and saving
run_analysis.summary_save(save=False,
                          obtain_dp=True, # obtain the mean and max diffraction pattern for 
                          log_scale_dp=True)